# N-Puzzle

In [ ]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import heapq

# Define an action as swapping two positions
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

## Helper functions

In [ ]:
# Generate all valid moves for the blank tile
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

# Apply an action to transition to a new state
def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

# Check if the puzzle is solvable
def is_solvable(state: np.ndarray) -> bool:
    flat_state = state.flatten()
    inversions = sum(
        1
        for i in range(len(flat_state))
        for j in range(i + 1, len(flat_state))
        if flat_state[i] > flat_state[j] > 0
    )
    blank_row = PUZZLE_DIM - np.where(state == 0)[0][0]
    if PUZZLE_DIM % 2 == 1:
        return inversions % 2 == 0
    else:
        if blank_row % 2 == 0:
            return inversions % 2 == 1
        else:
            return inversions % 2 == 0

# Manhattan distance heuristic
def hn(state: np.ndarray) -> int:
    h = 0
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            if state[i, j] != 0:
                x, y = divmod(state[i, j] - 1, PUZZLE_DIM)
                h += abs(x - i) + abs(y - j)
    return h  

while True:
    RANDOMIZE_STEPS = 100_000
    state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
        state = do_action(state, choice(available_actions(state)))
    if is_solvable(state):
        break

## A* search

In [ ]:
# Solution based on A* search
def astar_search(state: np.ndarray) -> None:
    goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    print("Initial state:")
    print(state, "\n")
    quality = 0
    g = 0

    open_list = [] # Priority queue
    state_tuple = tuple(state.flatten()) 
    heapq.heappush(open_list, (0, state_tuple))
    visited = set() # Visited states

    while open_list:
        f, state_tuple = heapq.heappop(open_list) 
        current_state = np.array(state_tuple).reshape((PUZZLE_DIM, PUZZLE_DIM))
        goal_state = np.array(goal_state).reshape((PUZZLE_DIM, PUZZLE_DIM))

        if np.array_equal(current_state, goal_state): # If goal state is equal to current state end loop
            eff = quality / g
            print("Final state:")
            print(current_state)
            print(f'Cost: {g}, Quality: {quality}, Efficiency: {eff}')
            return

        if state_tuple in visited:
            continue
        visited.add(state_tuple) # Mark the current state as visited
        quality += 1

        for action in available_actions(current_state): #Cosider all possible actions
            new_state = do_action(current_state, action)
            new_state_tuple = tuple(new_state.flatten()) 
            if new_state_tuple in visited:
                continue

            g = g + 1  
            h = hn(new_state)  
            f = g + h 
            heapq.heappush(open_list, (f, new_state_tuple))

# Not converting np.array to tuple for the state, causes errors as it is not hashable.
    # Credits for np.array -> tuple conversion in while loop: https://www.redblobgames.com/pathfinding/a-star/implementation.html
# Solve
astar_search(state)
